In [1]:
import pandas as pd

health_df = pd.read_csv('health_df.csv')
health_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34447 entries, 0 to 34446
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   location       34447 non-null  object
 1   client_gender  34447 non-null  int64 
 2   client_age     34447 non-null  int64 
 3   question       34447 non-null  object
 4   answer         34447 non-null  object
 5   category01     34447 non-null  object
 6   category02     34447 non-null  object
dtypes: int64(2), object(5)
memory usage: 1.8+ MB


In [2]:
from sentence_transformers import SentenceTransformer, util

# S-BERT 모델 로드
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

C:\Users\user\AppData\Roaming\Python\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
from tqdm import tqdm

# tqdm의 기본 설정
tqdm.pandas(desc="public embedding is creating")

# 모든 질문의 임베딩을 미리 계산하여 저장 (S-BERT로 임베딩 생성)
health_df['embedding'] = health_df['question'].progress_apply(lambda x: model.encode(x))

public embedding is creating: 100%|██████████| 34447/34447 [12:14<00:00, 46.88it/s]


In [4]:
# 사용자 입력에 대한 최적의 답변 선택 함수
def get_best_response(user_input, threshold=0.8):
    user_embedding = model.encode(user_input)  # S-BERT로 입력 문장 임베딩
    
    best_similarity = 0
    best_response = None
    
    for _, row in health_df.iterrows():
        # S-BERT로 코사인 유사도 계산
        similarity = util.cos_sim(user_embedding, row['embedding'])[0][0].item()
        if similarity > best_similarity:
            best_similarity = similarity
            best_response = row['answer']
    
    # 유사도가 기준 이상일 경우 응답 반환
    if best_similarity >= threshold:
        return best_response
    else:
        return "죄송하지만, 이해할 수 없습니다."

In [5]:
model.save('health_sbert_model2')

In [6]:
health_df.to_csv('health_df_embeddings2.csv', index=False)

In [7]:
# 사용자로부터 입력받기
while True:
    user_input = input("사용자 입력 (종료하려면 'exit' 입력): ")
    if user_input.lower() == 'exit':
        break
    
    # 가장 적합한 응답 가져오기
    response = get_best_response(user_input)
    print(f"챗봇: {response}")

챗봇: 눈병과 결막염이 의심되는 증상입니다. 잠시 기다려주시면 진료 도와드리겠습니다.
챗봇: 간단 접수 후 바로 도와드리겠습니다.
챗봇: 죄송하지만, 이해할 수 없습니다.
챗봇: 죄송하지만, 이해할 수 없습니다.
챗봇: 예약 내역 확인되었습니다. 2층 호흡기 내과로 가시면 됩니다.
챗봇: 목요일을 제외한 모든 요일에 진료 가능하십니다. 원하시는 날을 말씀해 주십시오.
챗봇: 네 날짜를 말씀해 주시면 비어 있는 시간대를 보여 드리겠습니다.
챗봇: 다음 달 진료는 전화문의를 통해 예약 가능합니다.
챗봇: 죄송하지만, 이해할 수 없습니다.
챗봇: 해당 날짜 예약이 불가능합니다. 다른 날짜를 말씀해 주시기 바랍니다.
챗봇: 피부 사정에 따라 다 달라서 말씀드리기가 어렵습니다. 증상은 사람마다 다릅니다.
챗봇: 예약일이 지났으면 다시 예약해 주셔야 합니다. 현재는 열 명의 대기 인원이 있습니다.
챗봇: 말씀을 이해하지 못했습니다. 다시 한번 말씀해 주시길 부탁드립니다.
챗봇: 안과 진료는 3층으로 엘리베이터 타고 올라가시면 됩니다.
챗봇: 새싹 약국에서 구입하실 수 있습니다. 요청 시 정확한 위치정보를 검색해 드리겠습니다.
챗봇: 즉시 수리 요청드리겠습니다. 기다려주시기 바랍니다.
챗봇: 수납은 현금 납부, 카드 결제 및 계좌 이체가 가능합니다.
챗봇: 죄송하지만, 이해할 수 없습니다.
챗봇: 죄송하지만, 이해할 수 없습니다.
